<a href="https://colab.research.google.com/github/cihandogan13/EfficientNetStyleTransfer/blob/master/EfficientnetStyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setup**

In [ ]:
import tensorflow as tf
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

**Functions**

In [ ]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

def load_img(path_to_img):
  max_dim = 1024
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)


def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)
  

**Image Files**

In [ ]:
content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')

# https://commons.wikimedia.org/wiki/File:Vassily_Kandinsky,_1913_-_Composition_7.jpg
style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

**Layers**

In [ ]:
content_layers = ["block7a_activation",
                  "block7b_activation",
                  "block7c_activation",
                  "block7d_activation"] 

style_layers = ["block1a_activation",
                  "block1b_activation",
                  "block1c_activation",
                  "block1d_activation",
                  "block2a_activation",
                  "block2b_activation",
                  "block2c_activation",
                  "block2d_activation",
                  "block2e_activation",
                  "block2f_activation",
                  "block2g_activation",
                  "block3a_activation",
                  "block3b_activation",
                  "block3c_activation",
                  "block3d_activation",
                  "block3e_activation",
                  "block3f_activation",
                  "block3g_activation",
                  "block4a_activation",
                  "block4b_activation",
                  "block4c_activation",
                  "block4d_activation",
                  "block4e_activation",
                  "block4f_activation",
                  "block4g_activation",
                  "block4h_activation",
                  "block4i_activation",
                  "block4j_activation",
                  "block5a_activation",
                  "block5b_activation",
                  "block5c_activation",
                  "block5d_activation",
                  "block5e_activation",
                  "block5f_activation",
                  "block5g_activation",
                  "block5h_activation",
                  "block5i_activation",
                  "block5j_activation",
                  "block6a_activation",
                  "block6b_activation",
                  "block6c_activation",
                  "block6d_activation",
                  "block6e_activation",
                  "block6f_activation",
                  "block6g_activation",
                  "block6h_activation",
                  "block6i_activation",
                  "block6j_activation",
                  "block6k_activation",
                  "block6l_activation",
                  "block6m_activation"]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)


def efnet_layers(layer_names):
  """ Creates a vgg model that returns a list of intermediate output values."""
  # Load our model. Load pretrained EfficientNetB7, trained on imagenet data
  efnet = tf.keras.applications.EfficientNetB7(include_top=False, weights='imagenet')
  efnet.trainable = False
  
  outputs = [efnet.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([efnet.input], outputs)
  return model

**Model Class**

In [ ]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.efnet =  efnet_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.efnet.trainable = False

  def call(self, inputs):
    "Expects float input in [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.efficientnet.preprocess_input(inputs)
    outputs = self.efnet(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}

**Initialization and Loss Functions**

In [ ]:
extractor = StyleContentModel(style_layers, content_layers)

style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']

def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)
    loss += total_variation_weight*tf.reduce_sum(tf.image.total_variation(image)) #tf.image.total_variation(image)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

**Weights and Optimizer**

In [ ]:
image = tf.Variable(content_image)
opt = tf.optimizers.Adam(learning_rate=0.010, beta_1=0.99, epsilon=1e3)
style_weight=2e8
content_weight=5e3
total_variation_weight=10
tf.image.total_variation(image).numpy()


**Process**

In [ ]:
import time
start = time.time()

epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='')
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))

end = time.time()
print("Total time: {:.1f}".format(end-start))

**Result**

In [ ]:
plt.subplot(1, 3, 1)
imshow(content_image, 'Content Image')

plt.subplot(1, 3, 2)
imshow(style_image, 'Style Image')

plt.subplot(1, 3, 3)
imshow(image, 'Final Image')

**Save Result**

In [ ]:
file_name = 'stylized-image.png'
tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)